In [1]:
from actionweaver.llms.openai.tools.tokens import TokenUsageTracker
from actionweaver.llms import patch
from openai import OpenAI

openai_client = patch(OpenAI())

In [2]:
import logging
import json
from pythonjsonlogger import jsonlogger

# Initialize logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create a StreamHandler for logging to the console
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)

# Define JSON format
log_format = jsonlogger.JsonFormatter(
    '%(asctime)s.%(msecs)04d %(levelname)s %(module)s %(funcName)s %(message)s %(lineno)d',
    datefmt='%Y-%m-%d %H:%M:%S'
)

console_handler.setFormatter(log_format)


In [3]:
from actionweaver import action

@action(name="GetCurrentTime", logger=logger)
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False, logger=logger)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


> /Users/tenghu/Code/actiontools/actionweaver/actions/action.py(92)__init__()
     90 
     91             pdb.set_trace()
---> 92             self.user_method = traceable(
     93                 self.name,
     94                 self.logger,



ipdb>  c


> /Users/tenghu/Code/actiontools/actionweaver/actions/action.py(92)__init__()
     90 
     91             pdb.set_trace()
---> 92             self.user_method = traceable(
     93                 self.name,
     94                 self.logger,



ipdb>  
ipdb>  c


In [13]:
get_current_time.logger

<Logger __main__ (DEBUG)>

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]


response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather],
    stream=False, 
    token_usage_tracker = TokenUsageTracker(5000),
    logger=logger,
)

response

{"asctime": "2023-12-29 18:28:01", "msecs": 291.0, "levelname": "INFO", "module": "helpers", "funcName": "wrapper", "message": "", "lineno": 67, "name": "actionwever_initial_chat_completion.chat.completions.create", "inputs": {"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}], "model": "gpt-3.5-turbo", "frequency_penalty": "NOT_GIVEN", "function_call": "NOT_GIVEN", "functions": "NOT_GIVEN", "logit_bias": "NOT_GIVEN", "logprobs": "NOT_GIVEN", "max_tokens": "NOT_GIVEN", "n": "NOT_GIVEN", "presence_penalty": "NOT_GIVEN", "response_format": "NOT_GIVEN", "seed": "NOT_GIVEN", "stop": "NOT_GIVEN", "stream": false, "temperature": "NOT_GIVEN", "tool_choice": "auto", "tools": [{"type": "function", "function": {"name": "GetWeather", "description": "Get the current weather in a given location", "parameters": {"properties": {"location": {"title": "Location"}, "unit": {"default": "fahrenheit",

Getting current weather


{"asctime": "2023-12-29 18:28:02", "msecs": 310.0, "levelname": "INFO", "module": "helpers", "funcName": "wrapper", "message": "", "lineno": 67, "name": "actionwever_initial_chat_completion.chat.completions.create", "inputs": {"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}, "ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_t5jODIwR3KX3W8aEGhTvD7mk', function=Function(arguments='{\\n  \"location\": \"San Francisco\"\\n}', name='GetWeather'), type='function')])", {"tool_call_id": "call_t5jODIwR3KX3W8aEGhTvD7mk", "role": "tool", "name": "GetWeather", "content": "{\"location\": \"San Francisco\", \"temperature\": \"72\", \"unit\": \"fahrenheit\"}"}], "model": "gpt-3.5-turbo", "frequency_penalty": "NOT_GIVEN", "function_call": "NOT_GIVEN", "functions": "NOT_GIVEN", "logit_bias": "NOT_GIVEN", "logprobs": "NOT

Getting current weather


{"asctime": "2023-12-29 18:28:03", "msecs": 27.0, "levelname": "INFO", "module": "helpers", "funcName": "wrapper", "message": "", "lineno": 67, "name": "actionwever_initial_chat_completion.chat.completions.create", "inputs": {"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}, "ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_t5jODIwR3KX3W8aEGhTvD7mk', function=Function(arguments='{\\n  \"location\": \"San Francisco\"\\n}', name='GetWeather'), type='function')])", {"tool_call_id": "call_t5jODIwR3KX3W8aEGhTvD7mk", "role": "tool", "name": "GetWeather", "content": "{\"location\": \"San Francisco\", \"temperature\": \"72\", \"unit\": \"fahrenheit\"}"}, "ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_4FQxtN78fwqujY3EqsFNLZ1N', function

ChatCompletion(id='chatcmpl-8bGYkMlWmKMiJHan2v7jjPzL8Rg2s', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The current weather in San Francisco is 72°F. I'm sorry, but I couldn't retrieve the current weather for Beijing.", role='assistant', function_call=None, tool_calls=None))], created=1703892482, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=155, total_tokens=182))